## DYLEMAT STUDENTA - WYTRAWNE WINO CZY TANIE I OBIAD?
### Wyznaczanie jakości wina na podstawie parametrów cieczy. | Aplikacja dla urzędu kontrolującego winiarnie



Celem projektu jest porównanie w zależności od problemu(regresja/klasyfikacja) różnego rodzaju modeli wraz z różnego rodzaju hiperparametrami.

● https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009
 
    
Team members:
- Monika Janicka
- Tomasz Kamiński
- Artur Kłapciński
- Elżbieta Kondracka-Zwolska

1. [BIBLIOTEKI](#BIBLIOTEKI)
2. [DANE_&_WYKRESY](#DANE_&_WYKRESY)
3. [PREPROCESSING](#PREPROCESSING)
4. [MODELE](#MODELE)
4.1 [MODEL1_REGRESJA_LINEARNA](##MODEL1_REGRESJA_LINIOWA)
4.2 [MODEL2_REGRESJA_LOGISTYCZNA](##MODEL2_REGRESJA_LOGISTYCZNA)
4.3 [MODEL3_DECISION_TREE](##MODEL3_DECISION_TREE)
4.4 [MODEL4_RANDOM_FOREST](##MODEL4_RANDOM_FOREST)
4.5 [MODEL5_GBOOST](##MODEL5_XGBOOST)
5. [PORÓWNANIE_MODELI](#PORÓWNANIE_MODELI)

<a id='BIBLIOTEKI'></a>
# 1. BIBLIOTEKI

In [ ]:
# Libraries for exploring, handling and visualizing data
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, plotly.express as px
import math
import sklearn   
import sklearn.datasets as datasets
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler    # Sklearn's preprocessing library  
from sklearn.linear_model import LinearRegression   
from sklearn import preprocessing
import sklearn.metrics
import sklearn.model_selection
import sklearn.tree as tree                                        # DRZEWO DECYZYJNE
from scipy import stats
from sklearn.model_selection import cross_validate, GridSearchCV       # DO WALIDACJI SKROŚNEJ(KRZYŻOWEJ)
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV   # Importing train and test data split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree    # DRZEWO DECYZYJNE
import matplotlib.pyplot as plt
%matplotlib inline

# Sklearn's metrics to evaluate our models
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score, f1_score, roc_auc_score,classification_report
from sklearn import metrics, tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier # Classifiers
from sklearn.linear_model import LogisticRegression
from mlxtend import plotting
# Setting theme style and color palette to seaborn 
sns.set_theme(context = 'notebook', style='darkgrid',palette='muted')

<a id='DANE'></a>
# 2. DANE_&_WYKRESY

In [ ]:
df = pd.read_csv("df_rm.csv") 
df # display the data frame
df.head(11)

<a id='PREPROCESSING'></a>
# 3. PREPROCESSING

<a id='MODELE'></a>
# 4. MODELE

<a id='MODEL1_REGRESJA_LINEARNA'></a>
## 4.1. MODEL1_REGRESJA_LINIOWA

<a id='MODEL2_REGRESJA_LOGISTYCZNA'></a>
## 4.2. MODEL2_REGRESJA_LOGISTYCZNA

<a id='MODEL3_DECISION_TREE'></a>
## 4.3. MODEL3_DECISION_TREE

<a id='MODEL4_RANDOM_FOREST'></a>
## 4.4. MODEL4_RANDOM_FOREST

<a id='MODEL5_XGBOOST'></a>
## 4.5. MODEL5_XGBOOST

<a id='PORÓWNANIE_MODELI'></a>
# 5. PORÓWNANIE_MODELI